# Création de cartes interactives relatives à la performance énergétique des logements et au niveau de vie des ménage (visualisation à l'échelle des communes d'Ile-de-France)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## A) Importation du dataframe Filosofi_dpe_IDF_com et quelques retraitements

In [ ]:
Filosofi_dpe_IDF_com = pd.read_csv("~/work/data_science_project/Filosofi_dpe_IDF_com.csv")

In [ ]:
# Calcul du ratio "nombre de DPE/nombre de ménages fiscaux":
Filosofi_dpe_IDF_com["ratio_DPE_menage"]=Filosofi_dpe_IDF_com["nombre_dpe"]/Filosofi_dpe_IDF_com["NBMENFISC16"]
Filosofi_dpe_IDF_com2=Filosofi_dpe_IDF_com[Filosofi_dpe_IDF_com["ratio_DPE_menage"]>0.05].reset_index()

In [ ]:
# 1) Codage de la variable "proportions de logements énergivores":
Filosofi_dpe_IDF_com2["proportion_energivores"]=Filosofi_dpe_IDF_com2.loc[:,['classe_consommation_energie_F','classe_consommation_energie_G']].sum(axis=1)

In [ ]:
# Fonction qui calcule la classe d'énergie majoritaire dans une commune
def maj(x):
    dic = 'A B C D E F G'.split()
    list = [x.classe_consommation_energie_A, x.classe_consommation_energie_B, x.classe_consommation_energie_C, 
            x.classe_consommation_energie_D, x.classe_consommation_energie_E, x.classe_consommation_energie_F,
            x.classe_consommation_energie_G]
    indice = [i for i, item in enumerate(list) if item == max(list)]
    if len(indice) != 0:
        return([ dic[j] for j in indice][len(indice)-1])
    else:
        return 'N'

In [ ]:
# Création d'une colonne avec ce maximum
Filosofi_dpe_IDF_com2['CE_Maj'] = Filosofi_dpe_IDF_com2.apply(lambda z: maj(z),  axis=1)

## B) Téléchargement des coordonnées géographiques des communes d'Ile-de-France:

In [ ]:
!pip install --upgrade botocore==1.23.26  
!pip install --upgrade urllib3==1.22.0 
!pip install py7zr 
!pip install s3fs 
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .

In [ ]:
import cartiflette.s3

communes_IDF = cartiflette.s3.download_vectorfile_url_all(
    values = ["75","91","92","93","94","95","77","78"],
    level="COMMUNE",
    vectorfile_format="geojson",
    decoupage="departement",
    year=2022)

In [ ]:
communes_IDF = communes_IDF[['NOM','INSEE_COM','geometry']]

In [ ]:
communes_IDF['INSEE_COM'] = communes_IDF['INSEE_COM'].apply(lambda n: int(n))

In [ ]:
# Jointure avec le dataframe Filosofi_dpe_IDF_com2:

Filosofi_dpe_IDF_com3=pd.merge(communes_IDF,Filosofi_dpe_IDF_com2,left_on="INSEE_COM",right_on="CODGEO",how="left")

In [ ]:
Filosofi_dpe_IDF_com4 = Filosofi_dpe_IDF_com3.dropna(subset=["geometry"])

## C) Création des cartes interactives

In [ ]:
pip install folium

In [ ]:
# Calcul des points centraux des communes afin de centrer la carte
data_geo2 = communes_IDF[['INSEE_COM', 'geometry']]
data_geo2['Center_point'] = data_geo2['geometry'].centroid

In [ ]:
#Extraction de la latitude et de la longitude de ces points centraux
data_geo2["lat"] = data_geo2.Center_point.map(lambda p: p.x)
data_geo2["long"] = data_geo2.Center_point.map(lambda p: p.y)

In [ ]:
# Première visualisation

import folium

# Calcul du centre de la carte
data_geo2 = communes_IDF[['INSEE_COM', 'geometry']]
data_geo2['Center_point'] = data_geo2['geometry'].centroid
#Extract lat and lon from the centerpoint
data_geo2["lat"] = data_geo2.Center_point.map(lambda p: p.x)
data_geo2["long"] = data_geo2.Center_point.map(lambda p: p.y)

# Sortie de la carte primaire
m = folium.Map(location=[data_geo2['long'].mean(), data_geo2['lat'].mean()],
               zoom_start=9, tiles='cartodbpositron')
m 
# (Visualisation non nécessaire)

In [ ]:
# @Marc: il ne manquerait pas la suite du code pour produire les cartes intéractives?